In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------
# Load Open Interest Data
# ------------------------------
oi_df = pd.read_csv("C:/Users/jay.jain/Desktop/Codes - Jay/Assignment 1/YBA OI.csv", skiprows=[0])
oi_df.rename(columns={oi_df.columns[0]: "Date", oi_df.columns[1]: "Open Interest"}, inplace=True)
oi_df["Date"] = pd.to_datetime(oi_df["Date"], errors='coerce')
oi_df["Open Interest"] = pd.to_numeric(oi_df["Open Interest"], errors='coerce')
oi_df = oi_df[["Date", "Open Interest"]].dropna()
oi_df = oi_df[(oi_df["Date"].dt.year >= 2012) & (oi_df["Date"].dt.year <= 2015)]
oi_df = oi_df.set_index("Date").resample("W").last().dropna().reset_index()

# ------------------------------
# Load Price & Volume Data
# ------------------------------
price_df = pd.read_csv("C:/Users/kavy.uprariya/Desktop/Codes - Kavy/Assignment 1/YBAc1.csv")
price_df["Date"] = pd.to_datetime(price_df["Date-Time"], errors='coerce').dt.tz_localize(None)
price_df = price_df[["Date", "Last", "Volume"]].rename(columns={"Last": "Price"}).dropna()
price_df = price_df[(price_df["Date"].dt.year >= 2012) & (price_df["Date"].dt.year <= 2015)]
price_df = price_df.set_index("Date").resample("W").last().dropna().reset_index()

# ------------------------------
# Merge and Calculate
# ------------------------------
merged = pd.merge(oi_df, price_df, on="Date", how="inner")
merged["OI Change (%)"] = merged["Open Interest"].pct_change() * 100
merged["Price Change (%)"] = merged["Price"].pct_change() * 100
merged["OI_Volume_Ratio"] = merged["Open Interest"] / merged["Volume"]
merged.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
merged.dropna(inplace=True)

# ------------------------------
# Divergence Zone Identification
# ------------------------------
merged["Bullish Divergence"] = (merged["Price Change (%)"] < 0) & (merged["OI_Volume_Ratio"].pct_change() > 0)
merged["Bearish Divergence"] = (merged["Price Change (%)"] > 0) & (merged["OI_Volume_Ratio"].pct_change() < 0)

# ------------------------------
# Plot: Price + OI/Volume with Divergences
# ------------------------------
fig, ax1 = plt.subplots(figsize=(14, 6))
ax1.set_title("Price vs OI/Volume Ratio (Weekly) with Divergence Zones", fontsize=15)
ax1.set_xlabel("Date")
ax1.set_ylabel("Price", color='tab:blue')
ax1.plot(merged["Date"], merged["Price"], color='tab:blue', label="Price", linewidth=2)
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel("OI/Volume Ratio", color='tab:red')
ax2.plot(merged["Date"], merged["OI_Volume_Ratio"], color='tab:red', linestyle='--', label="OI/Volume Ratio", linewidth=2)
ax2.tick_params(axis='y', labelcolor='tab:red')

# Highlight Divergence Zones
for i in range(len(merged)):
    if merged.iloc[i]["Bullish Divergence"]:
        ax1.axvspan(merged.iloc[i]["Date"], merged.iloc[i]["Date"], color="lime", alpha=0.2)
    elif merged.iloc[i]["Bearish Divergence"]:
        ax1.axvspan(merged.iloc[i]["Date"], merged.iloc[i]["Date"], color="red", alpha=0.15)

# Legends & Grid
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
plt.grid(True)
plt.tight_layout()
plt.show()